In [1]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing-sitting',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 5의 배수인 경우 건너뛰기
        if user_ID.startswith('SA') or user_ID.startswith('SE'):
            numeric_part = user_ID[2:]
            if numeric_part.isdigit() and int(numeric_part) % 3 == 0:
                continue

        for file_name in os.listdir(dir_path):
            if file_name.endswith('.txt'):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D04, D05, D07, F05, F06인 경우에만 처리
                if activity in ['D01', 'D04', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 0:  # 10의 배수인 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'label': label
                                    }
                                    record_count += 1
                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

print(df)

       user_ID trial_ID task_ID         gx         gy         gz   
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375  \
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   
2         SA01      R01     D01  31.860352 -22.216797   8.056641   
3         SA01      R01     D01   2.624512 -11.352539  29.052734   
4         SA01      R01     D01   7.263184  15.869141  26.184082   
...        ...      ...     ...        ...        ...        ...   
225397    SE14      R06     D07  -3.479004   2.563477  -0.061035   
225398    SE14      R06     D07  -2.197266   3.234863   0.488281   
225399    SE14      R06     D07  -4.394531   2.990723   0.549316   
225400    SE14      R06     D07  -2.746582   2.563477   0.000000   
225401    SE14      R06     D07  -4.028320   2.258301   0.122070   

                   label  
0                walking  
1                walking  
2                walking  
3                walking  
4                walking  
...                  

In [2]:
print(df['user_ID'].unique())

['SA01' 'SA02' 'SA04' 'SA05' 'SA07' 'SA08' 'SA10' 'SA11' 'SA13' 'SA14'
 'SA16' 'SA17' 'SA19' 'SA20' 'SA22' 'SA23' 'SE01' 'SE02' 'SE04' 'SE05'
 'SE07' 'SE08' 'SE10' 'SE11' 'SE13' 'SE14']


In [3]:
# CSV 파일로 저장
csv_path = '../data/SisFall_train.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_train.csv


In [4]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing-sitting',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 5의 배수인 경우에만 처리
        if (user_ID.startswith('SA') or user_ID.startswith('SE')) and user_ID[2:].isdigit() and int(user_ID[2:]) % 5 == 0:
            for file_name in os.listdir(dir_path):
                if file_name.endswith('.txt'):
                    # 파일 경로
                    file_path = os.path.join(dir_path, file_name)

                    # 파일 이름에 대한 정보
                    activity, _, trial_ID_with_extension = file_name.split('_')
                    trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                    # task_id가 D01, D04, D05, D07, F05, F06인 경우에만 처리
                    if activity in ['D01', 'D04', 'D05', 'D07', 'F06']:
                        # task_id에 해당하는 label 정보
                        task_ID = activity
                        label = label_mapping[task_ID]

                        # Angular velocity 변환을 위한 공식에 사용되는 값들
                        Range = 2000  # 예시 값을 사용
                        Resolution = 16  # 예시 값을 사용

                        # 파일에서 데이터를 읽어와서 처리
                        with open(file_path, 'r', encoding='utf-8') as file:
                            lines = file.readlines()

                        # 데이터 파싱 및 계산
                        gx_values = []
                        gy_values = []
                        gz_values = []

                        # 데이터 파싱 및 DataFrame에 추가
                        for i, line in enumerate(lines):
                            if i % 10 == 0:  # 10의 배수인 행만 처리
                                # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                                fields = line.strip().split(',')

                                if len(fields) >= 6:  # 필드 수 확인
                                    # gx, gy, gz 값 얻기 및 변환 계산
                                    gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                    gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                    gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                    gx_values.append(gx_value)
                                    gy_values.append(gy_value)
                                    gz_values.append(gz_value)

                                    if len(gx_values) == 1:
                                        # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                        df.loc[len(df)] = {
                                            'user_ID': user_ID,
                                            'trial_ID': trial_ID,
                                            'task_ID': task_ID,
                                            'gx': gx_values[0],
                                            'gy': gy_values[0],
                                            'gz': gz_values[0],
                                            'label': label
                                        }
                                        record_count += 1
                                        gx_values = []
                                        gy_values = []
                                        gz_values = []

print(df)

      user_ID trial_ID task_ID        gx         gy        gz   
0        SA05      R01     D01  0.183105 -25.634766 -7.934570  \
1        SA05      R01     D01 -1.831055 -17.944336 -8.178711   
2        SA05      R01     D01 -5.065918 -12.817383 -5.981445   
3        SA05      R01     D01 -2.624512  -8.361816 -6.103516   
4        SA05      R01     D01 -3.601074  -9.399414 -1.586914   
...       ...      ...     ...       ...        ...       ...   
57896    SE15      R05     D07 -1.281738   3.601074 -0.366211   
57897    SE15      R05     D07 -1.586914   3.295898 -0.610352   
57898    SE15      R05     D07 -1.586914   3.173828 -0.915527   
57899    SE15      R05     D07 -1.525879   2.868652 -0.976562   
57900    SE15      R05     D07 -1.281738   3.051758 -0.915527   

                  label  
0               walking  
1               walking  
2               walking  
3               walking  
4               walking  
...                 ...  
57896  standing-sitting  
57897  stan

In [ ]:
print(df['user_ID'].unique())

In [6]:
# CSV 파일로 저장
csv_path = '../data/SisFall_test.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_test.csv


In [19]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 0:  # 1, 11, 21, 31번째 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'label': label
                                    }
                                    record_count += 1
                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

print(df)

       user_ID trial_ID task_ID         gx         gy         gz     label
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375   walking
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   walking
2         SA01      R01     D01  31.860352 -22.216797   8.056641   walking
3         SA01      R01     D01   2.624512 -11.352539  29.052734   walking
4         SA01      R01     D01   7.263184  15.869141  26.184082   walking
...        ...      ...     ...        ...        ...        ...       ...
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211  standing
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352  standing
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527  standing
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562  standing
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527  standing

[331802 rows x 7 columns]


In [20]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]

print(df.head())

  user_ID trial_ID task_ID         gx         gy         gz    label   
0    SA01      R01     D01  -1.098633 -30.761719 -21.484375  walking  \
1    SA01      R01     D01 -10.864258 -46.752930  -3.173828  walking   
2    SA01      R01     D01  31.860352 -22.216797   8.056641  walking   
3    SA01      R01     D01   2.624512 -11.352539  29.052734  walking   
4    SA01      R01     D01   7.263184  15.869141  26.184082  walking   

   scaled_gx  scaled_gy  scaled_gz  
0   0.015592   0.010843   0.042287  
1   0.001771  -0.010123   0.066825  
2   0.062238   0.022046   0.081875  
3   0.020861   0.036290   0.110011  
4   0.027426   0.071980   0.106167  


In [21]:
# CSV 파일로 저장
csv_path = '../data/SisFall_train.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_train.csv


In [ ]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 3:
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'label': label
                                    }
                                    record_count += 1
                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

print(df)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]

print(df.head())